# Test multihead coherent state generation by pullback

Use the multihead (2-head) gates in the phase space 
to create a network that represent a coherent state,
by starting from a Gaussian state and making a pullback

<img src="../img/gaussianpullback.png" width="400" height="800" />

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>

nonlinearxwaves@gmail.com<br>
@created 22 july 2020<br>
@version 21 sep 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

In [4]:
tf.keras.backend.clear_session()

In [5]:
# reload the ps module (may delete this)
from importlib import reload
reload(ps)

<module 'thqml.phasespace' from '/home/claudio/tmp/thqml1/thqml-1.0/thqml/phasespace.py'>

## Dimension

In [6]:
N = 10

## Build vacuum by the Gaussian state

### Covariance matrix

In [7]:
g=np.eye(N)

### Displacement vector

In [8]:
d=np.zeros((N,1))

### Build the layer

In [9]:
vacuum = ps.VacuumLayer(N)

## Build the pullback layer

### Target displacement vector
The displacement vector is a column vector

In [10]:
dtarget = 3.0*np.ones((N,1))

In [11]:
dtarget.shape

(10, 1)

### Symplectic operator
A simple identity matrix

In [12]:
M = np.eye(N)



### Build the linear layer
The layer is a constant layer, as no training is needed

In [13]:
G = ps.DisplacementLayerConstant(dtarget)

## Build the pullback model

Input layers

In [14]:
xin = tf.keras.layers.Input(N)

In [15]:
x1, a1 = G(xin)
chir, chii = vacuum(x1, a1)

In [16]:
pullback = tf.keras.Model(inputs = xin, outputs=[chir, chii])

## Test the first derivative of the model
Evaluate the expectation value of the displacement as derivative of the characterisc function

In [17]:
x = tf.constant(np.zeros((1,N)), dtype=tf_real) # the derivative are evaluated at x=0
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    chir, chii = pullback(x)

Derivative of chir<br>
(must be zero)

In [18]:
print(tape.gradient(chir,x))

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)


Derivative of chii<br>
(this is the target d vector dtarget)

In [19]:
print(tape.gradient(chii,x))

tf.Tensor([[3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]], shape=(1, 10), dtype=float32)


# Test second derivative of the model
See https://www.tensorflow.org/guide/advanced_autodiff

Test the evaluation of the Hessian

In [20]:
x = tf.constant(np.zeros((1,N)), dtype=tf_real) # the derivative are evaluated at x=0
with tf.GradientTape(persistent=True) as t1:
    t1.watch(x)
    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        chir, chii = pullback(x)
    dchii_dx=t2.gradient(chii,x)
    dchir_dx=t2.gradient(chir,x)
d2chii_dx2=t1.jacobian(dchii_dx,x)
d2chir_dx2=t1.jacobian(dchir_dx,x)

In [21]:
dchii_dx.shape

TensorShape([1, 10])

In [22]:
d2chir_dx2.shape

TensorShape([1, 10, 1, 10])

Reshape to NxN

In [23]:
d2_mat=tf.reshape(d2chir_dx2,[N,N])

Apply the formula to evaluate g

In [24]:
-2*(d2_mat+tf.matmul(dchii_dx, dchii_dx, transpose_a=True))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[ 1., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.,  1., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0.,  1., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0.,  1.]], dtype=float32)>